In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file,'rb')as f:
    full_data=pickle.load(f)
    train_dataset=full_data['train_dataset']
    train_labels=full_data['train_labels']
    test_dataset=full_data['test_dataset']
    test_labels=full_data['test_labels']
    valid_dataset=full_data['valid_dataset']
    valid_labels=full_data['valid_labels']
del full_data
print("train_dataset  ",train_dataset.shape,train_labels.shape)
print("test_dataset ",test_dataset.shape,test_labels.shape)
print("valid_dataset ",valid_dataset.shape,valid_labels.shape)

train_dataset   (200000, 28, 28) (200000,)
test_dataset  (10000, 28, 28) (10000,)
valid_dataset  (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


# fit computation in graph

In [4]:
train_subset=1000
graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.constant(train_dataset[:train_subset,:])
    tf_train_labels=tf.constant(train_labels[:train_subset,:])
    tf_test_dataset=tf.constant(test_dataset)
    tf_valid_dataset=tf.constant(valid_dataset)
    
    
    #setting Parameter...............
    weights=tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases=tf.Variable(tf.zeros([num_labels]))
    
    #training..........................
    logits=tf.matmul(tf_train_dataset,weights) + biases
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimiser=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Prediction after training............
    train_prediction=tf.nn.softmax(logits)
    test_prediction=tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)
    valid_prediction=tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    

# Iterate the computation graph and see Accuracy

In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

    

In [6]:
num_step=1001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_step):
        opt,l,pred=sess.run([optimiser,loss,train_prediction])
        if(step % 100==0):
            print("loss at %d is %f" %(step,l))
            print("Training Accuracy now:%.2f%%" %accuracy(pred,train_labels[:train_subset,:]))
            print("Validation Accuracy now:%.2f%%" %accuracy(valid_prediction.eval(),valid_labels))
    print("-"*20)
    print("Test Accuracy :%.2f%%" %accuracy(test_prediction.eval(),test_labels))
            

Initialized
loss at 0 is 17.262524
Training Accuracy now:14.00%
Validation Accuracy now:14.61%
loss at 100 is 1.389235
Training Accuracy now:79.30%
Validation Accuracy now:66.85%
loss at 200 is 0.688968
Training Accuracy now:87.50%
Validation Accuracy now:67.74%
loss at 300 is 0.376706
Training Accuracy now:92.30%
Validation Accuracy now:68.33%
loss at 400 is 0.219824
Training Accuracy now:95.40%
Validation Accuracy now:68.70%
loss at 500 is 0.139361
Training Accuracy now:98.10%
Validation Accuracy now:69.04%
loss at 600 is 0.095987
Training Accuracy now:99.20%
Validation Accuracy now:69.32%
loss at 700 is 0.071389
Training Accuracy now:99.50%
Validation Accuracy now:69.47%
loss at 800 is 0.056193
Training Accuracy now:99.80%
Validation Accuracy now:69.65%
loss at 900 is 0.046253
Training Accuracy now:99.80%
Validation Accuracy now:69.76%
loss at 1000 is 0.039137
Training Accuracy now:99.80%
Validation Accuracy now:69.95%
--------------------
Test Accuracy :77.58%


# Using SGD

In [7]:
graph=tf.Graph()
batch_size=128
with graph.as_default():
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #parameters..............
    weights=tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases=tf.Variable(tf.zeros([num_labels]))
    
    #model computation ..................
    logits=tf.matmul(tf_train_dataset,weights)+biases
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
    
    #optimize............
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #prediction..........
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    test_prediction=tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)

In [8]:
num_step=3001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized ","-"*20)
    for step in range(num_step):
        #create feed_dict ................
        offset=np.random.randint(0,200000-128)
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        #run the session...............
        opt,l,pred_train=sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step%100 == 0):
            print("loss at %d:%f" %(step,l))
            print("Mini Batch Accuracy:%.2f%%" %accuracy(pred_train,batch_labels))
            print("Validation Accuracy:%.2f%%" %accuracy(valid_prediction.eval(),valid_labels))
    print("-"*30)
    print("Test Accuracy:%.2f%%" %accuracy(test_prediction.eval(),test_labels))
    
    

Initialized  --------------------
loss at 0:19.243279
Mini Batch Accuracy:10.94%
Validation Accuracy:10.28%
loss at 100:2.680439
Mini Batch Accuracy:68.75%
Validation Accuracy:70.02%
loss at 200:2.304486
Mini Batch Accuracy:67.19%
Validation Accuracy:73.24%
loss at 300:1.584449
Mini Batch Accuracy:75.78%
Validation Accuracy:74.18%
loss at 400:1.501973
Mini Batch Accuracy:72.66%
Validation Accuracy:74.23%
loss at 500:1.114059
Mini Batch Accuracy:80.47%
Validation Accuracy:75.38%
loss at 600:1.160684
Mini Batch Accuracy:82.03%
Validation Accuracy:75.85%
loss at 700:1.166031
Mini Batch Accuracy:80.47%
Validation Accuracy:75.82%
loss at 800:2.049074
Mini Batch Accuracy:65.62%
Validation Accuracy:76.04%
loss at 900:1.138509
Mini Batch Accuracy:78.91%
Validation Accuracy:76.09%
loss at 1000:1.406076
Mini Batch Accuracy:71.88%
Validation Accuracy:76.54%
loss at 1100:1.205722
Mini Batch Accuracy:78.12%
Validation Accuracy:76.28%
loss at 1200:1.303012
Mini Batch Accuracy:81.25%
Validation Accur

# Problem 1

In [12]:
image_size=28
num_labels=10
batch_size=128
K=1024

#Initialization..............................
graph=tf.Graph()
with graph.as_default():
    X=tf.placeholder(tf.float32,shape=(None,image_size*image_size))
    Y=tf.placeholder(tf.float32,shape=(None,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #parameters...................
    ##### Layer1
    w1=tf.Variable(tf.truncated_normal([image_size*image_size,K]))
    b1=tf.Variable(tf.zeros([K]))
    
    ##### Final Layer
    w2=tf.Variable(tf.truncated_normal([K,num_labels]))
    b2=tf.Variable(tf.zeros([num_labels]))
    
    
    
    #model Computation....................................
    y1=tf.nn.relu(tf.matmul(X,w1)+b1)
    logits =tf.matmul(y1,w2)+b2
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y))
    
    #optimize...........
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #prediction...................
    train_prediction=tf.nn.softmax(logits)
    
    t1=tf.nn.relu(tf.matmul(test_dataset,w1)+b1)
    t_logits=tf.matmul(t1,w2)+b2
    test_prediction=tf.nn.softmax(t_logits)
    
    v1=tf.nn.relu(tf.matmul(valid_dataset,w1)+b1)
    v_logits=tf.matmul(v1,w2)+b2
    valid_prediction=tf.nn.softmax(v_logits)

In [13]:
num_step=5001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized ","-"*20)
    for step in range(num_step):
        #create feed_dict ................
        offset=np.random.randint(0,200000-128)
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        
        #run the session...............
        opt,l,pred_train=sess.run([optimizer,loss,train_prediction],feed_dict={X:batch_data,Y:batch_labels})
        if (step%500 == 0):
            print("loss at %d:%f" %(step,l))
            print("Mini Batch Accuracy:%.2f%%" %accuracy(pred_train,batch_labels))
            print("Validation Accuracy:%.2f%%" %accuracy(valid_prediction.eval(),valid_labels))
    print("-"*30)
    
    print("Test Accuracy:%.2f%%" %accuracy(test_prediction.eval(),test_labels))

Initialized  --------------------
loss at 0:372.035278
Mini Batch Accuracy:3.91%
Validation Accuracy:27.34%
loss at 500:16.649914
Mini Batch Accuracy:78.12%
Validation Accuracy:80.57%
loss at 1000:4.839578
Mini Batch Accuracy:80.47%
Validation Accuracy:81.63%
loss at 1500:7.206089
Mini Batch Accuracy:82.81%
Validation Accuracy:80.67%
loss at 2000:4.810598
Mini Batch Accuracy:83.59%
Validation Accuracy:82.58%
loss at 2500:4.005328
Mini Batch Accuracy:76.56%
Validation Accuracy:81.55%
loss at 3000:3.968112
Mini Batch Accuracy:87.50%
Validation Accuracy:82.47%
loss at 3500:7.853579
Mini Batch Accuracy:78.12%
Validation Accuracy:78.44%
loss at 4000:1.811989
Mini Batch Accuracy:87.50%
Validation Accuracy:83.20%
loss at 4500:1.792280
Mini Batch Accuracy:85.94%
Validation Accuracy:82.80%
loss at 5000:1.370371
Mini Batch Accuracy:89.06%
Validation Accuracy:83.15%
------------------------------
Test Accuracy:89.85%
